In [ ]:
# Train the CNN Emotion Regression Model with MIDI data#
# Last editted by Pu Zeng, 19/10/2023 #

In [1]:
import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
# import mido
import string
import numpy as np
# from utilis import get_pianoroll_data
import pickle
import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
# import mido
import string
import numpy as np
# from utilis import get_pianoroll_data
import pickle
from sklearn import preprocessing
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

from torch.utils.data import TensorDataset


In [6]:
!python ../Data_Preprocessing/Emotion_MIDI2.py ../Toy_Dataset/Lakh/ ./Processed_Data/

All content numbers is 1
Total number of muisc 20
Total number of muisc 41
Total number of muisc 61
Total number of muisc 82
Total number of muisc 102
Total number of muisc 123
Total number of muisc 143
Total number of muisc 164
Total number of muisc 184



100%|██████████| 1/1 [00:00<00:00, 29.41it/s]

0it [00:00, ?it/s]
1it [00:01,  1.11s/it]
2it [00:01,  1.26it/s]
3it [00:02,  1.44it/s]
4it [00:02,  1.44it/s]C:\Milestone 2\Milestone 2\696-Milestone\Data_Preprocessing\Emotion_MIDI2.py:73: RuntimeWarning: invalid value encountered in divide
  roll = (roll/count).T

5it [00:03,  1.54it/s]
6it [00:04,  1.51it/s]
7it [00:05,  1.36it/s]
8it [00:06,  1.18it/s]
9it [00:07,  1.01it/s]
10it [00:08,  1.00s/it]
11it [00:09,  1.14it/s]
12it [00:09,  1.44it/s]
13it [00:10,  1.41it/s]
14it [00:10,  1.37it/s]
15it [00:12,  1.08it/s]D:\programing\anaconda3\lib\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(

16it [00:12,  1.27it/s]
17it [00:13,  1.42it/s]
18it [00:13,  1.55it/s]
19it [00:14,  1.33it/s]
20it [00:15,  1.28it/s]
20it [00:15,  1.28it/s]

0it [00:00,

In [7]:
!python ../Data_Preprocessing/Emotion_Label.py ./Processed_Data/ ./Processed_Data/


  0%|          | 0/18 [00:00<?, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (817 > 512). Running this sequence through the model will result in indexing errors

100%|██████████| 18/18 [02:28<00:00,  8.25s/it]

100%|██████████| 21/21 [00:00<00:00, 27.21it/s]

100%|██████████| 19/19 [00:00<00:00, 21.33it/s]

100%|██████████| 21/21 [00:00<00:00, 32.74it/s]

100%|██████████| 19/19 [00:00<00:00, 37.08it/s]

100%|██████████| 19/19 [00:00<00:00, 22.26it/s]

100%|██████████| 18/18 [00:00<00:00, 22.41it/s]

100%|██████████| 20/20 [00:00<00:00, 25.53it/s]

100%|██████████| 18/18 [00:00<00:00, 34.27it/s]


In [2]:
file=open("./Processed_Data/lakh_emotion.bin","rb")
music_data = pickle.load(file) #保存list到文件
file.close()

In [3]:
# Show the labels
np.unique([x[1] for x in music_data])

array(['amazing', 'anger', 'angry', 'awe', 'emotional', 'excited', 'fear',
       'happy', 'hopeful', 'hurt', 'joy', 'joyous', 'love', 'negative',
       'positive', 'powerful', 'sad', 'sadness', 'strong', 'uneasy',
       'upset'], dtype='<U9')

In [4]:
# Selected the most common one word labels
from tqdm import tqdm
X = []
y = []
chosen_label = ['sadness', 'happy', 'love', 'joy', 'negative', 'positive', 'anger',
       'awe', 'emotional', 'hopeful', 'angry', 'excited', 'fear', 'hurt',
       'fearful']
# for i in tqdm(range(1,10)):
file=open("./Processed_Data/lakh_emotion.bin","rb")
music_data = pickle.load(file) #保存list到文件
file.close()
for m in music_data:
    if m[1] in chosen_label:
        X.append(m[0])
        if m[1] == 'anger':
            m[1] = 'angry'
        if m[1] == 'fearful':
            m[1] = 'fear'
        if m[1] == 'joy':
            m[1] = 'happy'
        y.append(m[1])
X = np.array(X)

In [5]:
y = np.array(y)
X_new = []
y_new = []
for l in np.unique(y):
    n = (y==l).sum()
    if n>=6:
        X_new += list(X[y==l])
        y_new += list(y[y==l])
X = X_new
y = y_new

In [6]:
# Transform labels into one-hot variable
from sklearn import preprocessing
import torch

le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

label_dict = {}
for cl in le.classes_:
    label_dict.update({cl:le.transform([cl])[0]})

In [7]:
num_labels = len(np.unique(y))

In [8]:
#Define CNN model
class CNNModel(nn.Module):
    def __init__(self, hiddenSize, outChannels, dropoutRate, activate):
        super().__init__()
        self.outChannels = outChannels
        self.activate = nn.Sigmoid() if activate == "Sigmoid" else nn.ReLU()
        self.conv1 = nn.Conv2d(1, 24, (10,1))
        self.pool = nn.MaxPool2d((2, 1))
        self.conv2 = nn.Conv2d(24, 48, (10,1))
        self.conv3 = nn.Conv2d(48, 96, (10,1))
        self.conv4 = nn.Conv2d(96, 192, (10,1))
        self.conv5 = nn.Conv2d(192, 384, (5,2))
        self.conv6 = nn.Conv2d(384, 192, (5,2))
        self.dense1 = nn.Linear(48384, hiddenSize)
        self.dropout = nn.Dropout(dropoutRate)
        self.dense2 = nn.Linear(hiddenSize, num_labels)

    def forward(self, x):
        x = self.pool(self.activate(self.conv1(x)))
        x = self.pool(self.activate(self.conv2(x)))
        # print(x.shape)
        x = self.dropout(self.pool(self.activate(self.conv3(x))))
        x = self.pool(self.activate(self.conv4(x)))
        x = self.pool(self.activate(self.conv5(x)))
        x = self.pool(self.activate(self.conv6(x)))
        # print(x.shape)
        x = x.view(-1, 48384)
        x = self.dropout(self.activate(self.dense1(x)))
        return self.dense2(x)

# Number of neurons in the first fully-connected layer
hiddenSize = 4096
# Number of feature filters in second convolutional layer
numFilters = 25
# Dropout rate
dropoutRate = 0.2
# Activation function
activation = "ReLU"
# Learning rate
learningRate = 0.005
# Momentum for SGD optimizer
momentum = 0.9
# Number of training epochs
numEpochs = 20

In [9]:
#Define Trian and Validation function

from tqdm import tqdm
def train_epoch(cnn,device,dataloader,loss_fn,optimizer):
    cnn.train()
    cnnRunningLoss = 0
    total = 0
    R2 = 0
    cnnCorrect=0
    total1=0
    for i, (inputs, labels) in enumerate(dataloader, 0):
        optimizer.zero_grad()
        inputs = inputs.to('cuda').reshape(-1,1,512,128)
        
        labels = labels.to('cuda')
        # Forward propagation
        cnnOutputs = cnn(inputs)
        # print(cnnOutputs.shape)
        l2_lambda = 0.001
        l2_reg = torch.tensor(0.).to('cuda')
        for param in cnn.parameters():
            l2_reg += torch.norm(param)
        # Backpropagation
        cnnLoss = criterion(cnnOutputs, labels)+l2_reg*l2_lambda
        cnnLoss.backward()
        # Gradient update
        optimizer.step()
        total += 1
        total1+=labels.size(0)
        cnnRunningLoss += cnnLoss.item()
        _, cnnPredicted = torch.max(cnnOutputs.data, 1)
        cnnCorrect += (cnnPredicted == labels).sum().item()
    return cnnRunningLoss/total, cnnCorrect/total1

def valid_epoch(cnn,device,dataloader,loss_fn):
    cnn.eval()
    totalLoss = 0
    total = 0
    total1 = 0
    R2 = 0
    cnnLoss = 0
    cnnCorrect=0
    for inputs, labels in dataloader:
        inputs = inputs.to('cuda').reshape(-1,1,512,128)
        labels = labels.to('cuda')
        cnnOutputs = cnn(inputs)
        cnnLoss = loss_fn(cnnOutputs, labels)
        _, cnnPredicted = torch.max(cnnOutputs.data, 1)
        total += labels.size(0)
        total1 +=1
        totalLoss += cnnLoss.item()
        cnnCorrect += (cnnPredicted == labels).sum().item()
    accuracy = cnnCorrect / total * 100
    cnn.train()
    return totalLoss/total1, accuracy

In [10]:
# Train the model
import sklearn
from sklearn.model_selection import StratifiedKFold
X_train_v, X_test, y_train_v, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) # First Split the dataset and use the training set for 5-fold validation

X_train_v = torch.as_tensor(X_train_v, dtype=torch.float) # an alternative to torch.from_numpy
y_train_v = torch.as_tensor(y_train_v, dtype=torch.float)
X_test = torch.as_tensor(X_test, dtype=torch.float)
y_test = torch.as_tensor(y_test, dtype=torch.float)
splits=StratifiedKFold(n_splits=5,shuffle=True,random_state=99)
train_dataset = TensorDataset(X_train_v, y_train_v)
test_dataset = TensorDataset(X_test, y_test)
batch_size=128
models = []
history = {'fold':[], 'train_loss': [], 'test_loss': [],'train_acc':[],'test_acc':[]}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
for fold, (train_idx,val_idx) in enumerate(splits.split(X_train_v,y_train_v)):
    print('Fold {}'.format(fold + 1))
    t = pd.DataFrame(y_train_v[train_idx], columns = ['class']).reset_index()
    idx = []
    # Resample to avoid unbalanced dataset
    y_train = []
    for i in range(0,num_labels):
      idx += list(t[t['class']==i].sample(100,replace=True)['index'])
      y_train += [i]*100
    X_train = X_train_v[train_idx][idx]

    t = pd.DataFrame(y_train_v[val_idx], columns = ['class']).reset_index()
    idx = []
    y_val = []
    for i in range(0,num_labels):
      # print(i)
      if i in t['class'].values:
          idx += list(t[t['class']==i].sample(20,replace=True)['index'])
          y_val += [i]*20
    X_val =  X_train_v[val_idx][idx]

    X_train = torch.as_tensor(X_train, dtype=torch.float) # an alternative to torch.from_numpy
    y_train = torch.as_tensor(y_train, dtype=torch.float).type(torch.LongTensor)
    X_val = torch.as_tensor(X_val, dtype=torch.float)
    y_val = torch.as_tensor(y_val, dtype=torch.float).type(torch.LongTensor)

    train_dataset = TensorDataset(X_train, y_train)
    test_dataset = TensorDataset(X_val, y_val)

    from torch.utils.data import Dataset, DataLoader
    train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)
    
    model = CNNModel(hiddenSize, numFilters, dropoutRate, activation).to('cuda')
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(list(model.parameters()), lr=learningRate, momentum=momentum)
    # optimizer = torch.optim.Adam(list(model.parameters()), lr=learningRate)
    best_test = -np.inf
    best_model = None
    
    for epoch in range(numEpochs):
        train_loss, train_R2=train_epoch(model,device,train_dataloader,criterion,optimizer)
        test_loss, test_R2=valid_epoch(model,device,test_dataloader,criterion)
        history['fold'].append(fold)
        history['train_loss'].append(train_loss)
        history['test_loss'].append(test_loss)
        history['train_acc'].append(train_R2)
        history['test_acc'].append(test_R2)   
        if test_R2>best_test:
            test_test = test_R2
            best_model = model
        print('Train Loss: {}, Train_acc: {}, Test Loss: {}, Test acc: {}'.format(train_loss, train_R2, test_loss, test_R2))
    models.append([best_model])

C:\Users\12364\AppData\Local\Temp\ipykernel_18004\2016658442.py:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:248.)
  X_train_v = torch.as_tensor(X_train_v, dtype=torch.float) # an alternative to torch.from_numpy


Fold 1
Train Loss: 0.748879685997963, Train_acc: 0.445, Test Loss: 0.6353520154953003, Test acc: 72.5
Train Loss: 0.6721135824918747, Train_acc: 0.545, Test Loss: 0.5202437043190002, Test acc: 72.5
Train Loss: 0.6057620868086815, Train_acc: 0.565, Test Loss: 0.40458256006240845, Test acc: 100.0
Train Loss: 0.5238330662250519, Train_acc: 0.845, Test Loss: 0.32948991656303406, Test acc: 100.0
Train Loss: 0.39969880878925323, Train_acc: 0.885, Test Loss: 0.12069686502218246, Test acc: 100.0
Train Loss: 0.3175176605582237, Train_acc: 0.87, Test Loss: 0.06911692023277283, Test acc: 100.0
Train Loss: 0.28870122879743576, Train_acc: 0.87, Test Loss: 0.07734281569719315, Test acc: 100.0
Train Loss: 0.2672156095504761, Train_acc: 0.92, Test Loss: 0.05601201206445694, Test acc: 100.0
Train Loss: 0.2213122472167015, Train_acc: 0.92, Test Loss: 0.050890274345874786, Test acc: 100.0
Train Loss: 0.2031673826277256, Train_acc: 0.92, Test Loss: 0.04554833844304085, Test acc: 100.0
Train Loss: 0.240481

In [14]:
# Caculate the average accuray
# X_train_v, X_test, y_train_v, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
t = pd.DataFrame(y_test, columns = ['class']).reset_index()
idx = []
y_test = []
for i in range(0,num_labels):
    if i in t['class'].values:
        idx += list(t[t['class']==i].sample(20,replace=True)['index'])
        y_test += [i]*20
X_test = np.array(X_test)
X_test =  X_test[idx]
X_test = torch.as_tensor(X_test, dtype=torch.float)
y_test = torch.as_tensor(y_test, dtype=torch.float).type(torch.LongTensor)
test_dataset = TensorDataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)
acc=[]
for best_model in models:
    acc.append(valid_epoch(best_model[0],device,test_dataloader,criterion)[1])

In [15]:
np.mean(acc)

51.0

In [16]:
# Calculate the accuray of the ensemble model
outputs = []    
for model in models:
  model[0].eval()
  model_output = []
  label_= []
  model[0].to('cpu')
  for inputs, labels in test_dataloader:
      inputs = inputs.to('cpu').reshape(-1,1,512,128)
      labels = labels.to('cpu')
      cnnOutputs = model[0](inputs)
      model_output.append(cnnOutputs)
      label_.append(labels)
  outputs.append(torch.vstack(model_output))
shape = outputs[0].shape
cnnresult = torch.vstack(outputs).reshape(-1,shape[0],shape[1]).mean(axis=0)
labels = np.hstack(label_)
_, cnnPredicted = torch.max(cnnresult.data, 1)
cnnCorrect = (cnnPredicted.detach().numpy() == labels).sum().item()
cnnCorrect/len(labels)*100

50.0

In [ ]:
# Draw the confusion matrix
from sklearn.metrics import confusion_matrix
confu_m = confusion_matrix(labels, cnnPredicted)
import seaborn as sns
sns.heatmap(confu_m,square=False)

In [ ]:
label_dict = {}
for cl in le.classes_:
    label_dict.update({cl:le.transform([cl])[0]})
label_dict

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
scores = pd.DataFrame(precision_recall_fscore_support(cnnPredicted, labels, labels = list(range(12))),index=['precision','recall','f1','support'], columns = list(label_dict)[:12])
scores